In [8]:
from imblearn.combine import SMOTEENN
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
import pandas as pd
import pickle
from scipy.stats import uniform, randint
from sklearn.inspection import permutation_importance


feature_imp_dict = {}

df = pd.read_csv('cleaned_dataset.csv',index_col=0)
X = df.drop(['Churn'],axis=1)
y = df['Churn']
# UpSampling
sm = SMOTEENN()
X_res, y_res = sm.fit_resample(X, y)

Xr_train, Xr_test, yr_train, yr_test = train_test_split(X_res, y_res, test_size=0.2)

# Initialize an empty list to store model scores
model_scores = []

# Create a list of models to evaluate
models = [
    ('Random Forest', RandomForestClassifier(random_state=42),
        {'n_estimators': [50, 100, 200],
         'max_depth': [None, 10, 20]}),  # Add hyperparameters for Random Forest
    ('Gradient Boosting', GradientBoostingClassifier(random_state=42),
        {'n_estimators': [50, 100, 200],
         'learning_rate': [0.05, 0.1, 0.5]}),  # Add hyperparameters for Gradient Boosting
    ('Support Vector Machine', SVC(random_state=42, class_weight='balanced'),
        {'C': [0.1, 1, 10],
         'gamma': ['scale', 'auto']}),  # Add hyperparameters for SVM
    ('Logistic Regression', LogisticRegression(random_state=42, class_weight='balanced'),
        {'C': [0.1, 1, 10],
         'penalty': ['l2']}),  # Add hyperparameters for Logistic Regression
    ('K-Nearest Neighbors', KNeighborsClassifier(),
        {'n_neighbors': [3, 5, 7],
         'weights': ['uniform', 'distance']}),  # Add hyperparameters for KNN
    ('Decision Tree', DecisionTreeClassifier(random_state=42),
        {'max_depth': [None, 10, 20],
         'min_samples_split': [2, 5, 10]}),  # Add hyperparameters for Decision Tree
    ('Ada Boost', AdaBoostClassifier(random_state=42),
        {'n_estimators': [50, 100, 200],
         'learning_rate': [0.05, 0.1, 0.5]}),  # Add hyperparameters for Ada Boost
    ('XG Boost', XGBClassifier(random_state=42),
        {'max_depth': randint(3, 6), 
         'learning_rate': uniform(0.01, 0.2),  
         'n_estimators': randint(100, 300),  
         'subsample': uniform(0.8, 0.2)}),  # Add hyperparameters for XG Boost
    ('Naive Bayes', GaussianNB(), {})  # No hyperparameters for Naive Bayes
]

best_model = None
best_accuracy = 0.0
best_f1 = 0.0
best_auc_roc = 0.0

# Iterate over the models and evaluate their performance
for name, model, param_grid in models:
    # Create a pipeline for each model
    pipeline = Pipeline([
        ('scaler', MinMaxScaler()),  # Feature Scaling
        ('model', model)
    ])

    # Hyperparameter tuning using RandomizedSearchCV for XG Boost
    if name == 'XG Boost':
        random_search = RandomizedSearchCV(estimator=model, param_distributions=param_grid,
                                           n_iter=100, cv=3, verbose=0, random_state=42, n_jobs=-1)
        random_search.fit(Xr_train, yr_train)
        pipeline = random_search.best_estimator_
    # Hyperparameter tuning using GridSearchCV for other models
    elif param_grid:
        grid_search = GridSearchCV(model, param_grid, cv=2, verbose=0)
        grid_search.fit(Xr_train, yr_train)
        pipeline = grid_search.best_estimator_

    # Fit the pipeline on the training data
    pipeline.fit(Xr_train, yr_train)

    # Make predictions on the test data
    y_pred = pipeline.predict(Xr_test)

    # Calculate accuracy score
    accuracy = accuracy_score(yr_test, y_pred)

    #Calculate f1 score
    f1 = f1_score(yr_test, y_pred)

    # Calculate AUC-ROC score
    auc_roc = roc_auc_score(yr_test, y_pred) 

    if  name in ['Random Forest', 'Gradient Boosting', 'Ada Boost', 'XG Boost', 'Decision Tree']:
        importances = pipeline.feature_importances_
        feature_names = X.columns
        feature_importances = pd.DataFrame(importances, index=feature_names, columns=['Importance']).sort_values('Importance', ascending=False)
        feature_imp_dict[name] = feature_importances

    elif name == 'Logistic Regression':
        coef = pipeline.coef_[0]
        feature_importances = pd.DataFrame(coef, index=X.columns, columns=['Importance']).sort_values('Importance', ascending=False)
        feature_imp_dict[name] = feature_importances

    elif name in ['K-Nearest Neighbors','Naive Bayes', 'Support Vector Machine']:    
        result = permutation_importance(pipeline, Xr_test, yr_test, n_repeats=10, random_state=42)
        feature_importances = pd.DataFrame(result.importances_mean, index=X.columns, columns=['Importance']).sort_values('Importance', ascending=False)
        feature_imp_dict[name] = feature_importances

    # Append model name and accuracy to the list
    model_scores.append({'Model': name, 'Accuracy': accuracy})

    # Convert the list to a DataFrame
    scores_df = pd.DataFrame(model_scores)

    # Print the performance metrics
    print("Model:", name)
    print("Test Accuracy:", round(accuracy, 3),"%")
    print("F1 Score:", round(f1,3))
    print("ROC AUC Score:", round(auc_roc,3))
    print()

    # Save the models to a pickle file
    with open(f'models/{name}.pkl','wb') as fp:
        pickle.dump(pipeline, fp)

    # Check if the current model has the best accuracy
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_f1 = f1
        best_auc_roc = auc_roc
        best_model = pipeline

# Retrieve the overall best model
print("Best Model:")
print("Test Accuracy:", best_accuracy)
print("F1 Score:", best_f1)
print("ROC AUC Score:", best_auc_roc)
print("Model Pipeline:", best_model, "with accuracy", round(best_accuracy, 2), "%")


Model: Random Forest
Test Accuracy: 0.958 %
F1 Score: 0.961
ROC AUC Score: 0.957

Model: Gradient Boosting
Test Accuracy: 0.966 %
F1 Score: 0.968
ROC AUC Score: 0.965

Model: Support Vector Machine
Test Accuracy: 0.964 %
F1 Score: 0.965
ROC AUC Score: 0.966



/home/ammar-asim/misc/shine/.venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/ammar-asim/misc/shine/.venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_

Model: Logistic Regression
Test Accuracy: 0.935 %
F1 Score: 0.939
ROC AUC Score: 0.934

Model: K-Nearest Neighbors
Test Accuracy: 0.982 %
F1 Score: 0.983
ROC AUC Score: 0.981

Model: Decision Tree
Test Accuracy: 0.94 %
F1 Score: 0.944
ROC AUC Score: 0.939



/home/ammar-asim/misc/shine/.venv/lib/python3.12/site-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/home/ammar-asim/misc/shine/.venv/lib/python3.12/site-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/home/ammar-asim/misc/shine/.venv/lib/python3.12/site-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/home/ammar-asim/misc/shine/.venv/lib/python3.12/site-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. U

Model: Ada Boost
Test Accuracy: 0.948 %
F1 Score: 0.953
ROC AUC Score: 0.947

Model: XG Boost
Test Accuracy: 0.962 %
F1 Score: 0.965
ROC AUC Score: 0.961

Model: Naive Bayes
Test Accuracy: 0.906 %
F1 Score: 0.912
ROC AUC Score: 0.906

Best Model:
Test Accuracy: 0.9819432502149613
F1 Score: 0.9832935560859188
ROC AUC Score: 0.9808917008257589
Model Pipeline: KNeighborsClassifier(n_neighbors=3, weights='distance') with accuracy 0.98 %


In [9]:
for i,j in feature_imp_dict.items():
    print(f"\nFeature Importance for {i}:")
    print('--------------------------------')
    print(j)
    print('--------------------------------\n')


Feature Importance for Random Forest:
--------------------------------
                  Importance
Contract            0.199789
tenure              0.180900
MonthlyCharges      0.101548
OnlineSecurity      0.100847
TotalCharges        0.100756
TechSupport         0.085109
InternetService     0.057598
OnlineBackup        0.039588
DeviceProtection    0.028089
PaymentMethod       0.018662
Dependents          0.015825
Partner             0.015394
StreamingTV         0.010856
StreamingMovies     0.010071
gender              0.009854
MultipleLines       0.008600
PaperlessBilling    0.007461
SeniorCitizen       0.005519
PhoneService        0.003535
--------------------------------


Feature Importance for Gradient Boosting:
--------------------------------
                  Importance
Contract            0.580647
tenure              0.121202
MonthlyCharges      0.090292
TotalCharges        0.085427
InternetService     0.067164
TechSupport         0.013331
OnlineSecurity      0.009196
Online